# Using `OllamaLLM`

In this notebook, we show how to use the `OllamaLLM` class.

__Requirements:__

1. Have `ollama` installed (see instructions in the official [README](https://github.com/ollama/ollama))
2. Ollama running in the background, via: `ollama serve`

In [1]:
# install dependencies
!pip install llm-agents-from-scratch -q


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Instantiating an `OllamaLLM` object

To create an `OllamaLLM` object, simply pass in the name of the model you'd like to use. For the list of all models, see the official [docs](https://ollama.com/search).

In [1]:
from llm_agents_from_scratch.llms import OllamaLLM

In [2]:
llm = OllamaLLM(
    model="llama3.2",
)

## Complete

We use the `.complete()` method to perform text completion with our `OllamaLLM` object. 

In [3]:
response = await llm.complete(prompt="Tell me a joke.")

In [4]:
type(response)

llm_agents_from_scratch.data_structures.llm.CompleteResult

In [5]:
print(response.response)

Why don't eggs tell jokes?

Because they'd crack each other up!


## Chat

We use the `.chat()` method to chat with `OllamaLLM` object, using the chat API.

In [6]:
user_message, response_message = await llm.chat("Tell me a joke.")

In [10]:
type(user_message)

llm_agents_from_scratch.data_structures.llm.ChatMessage

In [12]:
print(user_message)

role=<ChatRole.USER: 'user'> content='Tell me a joke.' tool_calls=None


In [11]:
type(response_message)

llm_agents_from_scratch.data_structures.llm.ChatMessage

In [13]:
print(response_message.content)

Why couldn't the bicycle stand up by itself?

Because it was two-tired! (get it?)


## Structured Output

We use the `.structured_output()` method to produce structured responses, represented as `~pydantic.BaseModel`, with our `OllamaLLM` object. Here, we'll ask it to produce a structured data class `Joke` that contains a `topic` and a `content` attribute.

In [14]:
from pydantic import BaseModel, Field


class Joke(BaseModel):
    """A structured representation of a joke."""

    topic: str = Field(description="Topic of the joke.")
    content: str = Field(description="Joke content.")

In [15]:
joke = await llm.structured_output(
    prompt="Tell me a new joke about any topic you like.",
    mdl=Joke,
)

In [16]:
joke

Joke(topic='space', content='Why did the astronaut break up with his girlfriend?')

## Tool Calling

In this section of the notebook, we demonstrate how to perform tool calls with an `OllamaLLM`. To do so, first we'll need to create some tools using the `llm-agents-from-scratch` library.

In [17]:
from pydantic import BaseModel

from llm_agents_from_scratch import (
    PydanticFunctionTool,
)

In [18]:
class AddOneParams(BaseModel):
    """Parameters for `add_one` tool."""

    x: float


def add_one(params: AddOneParams) -> int:
    """Adds one to a given number."""
    return params.x + 1

In [19]:
add_one_tool = PydanticFunctionTool(func=add_one)

The `parameters_json_schema` of a `BaseTool` object, shows how the tool's parameters will be passed to the LLM.

In [20]:
add_one_tool.parameters_json_schema

{'description': 'Parameters for `add_one` tool.',
 'properties': {'x': {'title': 'X', 'type': 'number'}},
 'required': ['x'],
 'title': 'AddOneParams',
 'type': 'object'}

### Testing the tool

NOTE: this is a direct invocation of the tool, without any LLM invocation. A `ToolCall` is how he bundle the the parameters that should be passed to the tool. The `ToolCall` object is also the parameter used in the `__call__` method for all `BaseTool` types.

In [21]:
from llm_agents_from_scratch.data_structures.tool import ToolCall

In [22]:
# add one
tool_call = ToolCall(
    tool_name=add_one_tool.name,
    arguments={"x": 3.15},
)

# invoke the __call__ method
add_one_tool(tool_call)

ToolCallResult(tool_call=ToolCall(tool_name='add_one', arguments={'x': 3.15}), content='4.15', error=False)

In [23]:
# add one
tool_call = ToolCall(
    tool_name=add_one_tool.name,
    arguments={"x": 3.15},
)
add_one_tool(tool_call)

ToolCallResult(tool_call=ToolCall(tool_name='add_one', arguments={'x': 3.15}), content='4.15', error=False)

### Get `OllamaLLM` to use the tool

In [25]:
user_message, response_message = await llm.chat(
    input="Add one to fifty-five point three. Use only the appropriate tools!",
    tools=[add_one_tool],
)

We can see that the LLM is requesting for a tool call for `add_one`.

In [26]:
response_message

ChatMessage(role=<ChatRole.ASSISTANT: 'assistant'>, content='', tool_calls=[ToolCall(tool_name='add_one', arguments={'x': 55.3})])

In the LLM agent processing cycle, we would next perform the tool call, and pass the result back to the LLM.

In [30]:
(
    tool_messages,
    response_message,
) = await llm.continue_chat_with_tool_results(
    tool_call_results=[
        add_one_tool(
            response_message.tool_calls[0],
        ),  # returns a ToolCallResult
    ],
    chat_history=[
        user_message,
        response_message,
    ],
)

In [31]:
tool_messages

[ChatMessage(role=<ChatRole.TOOL: 'tool'>, content='{\n    "tool_call": {\n        "tool_name": "add_one",\n        "arguments": {\n            "x": 55.3\n        }\n    },\n    "content": "56.3",\n    "error": false\n}', tool_calls=None)]

In [32]:
print(tool_messages[0].content)

{
    "tool_call": {
        "tool_name": "add_one",
        "arguments": {
            "x": 55.3
        }
    },
    "content": "56.3",
    "error": false
}


In [33]:
response_message

ChatMessage(role=<ChatRole.ASSISTANT: 'assistant'>, content='I used the `add_one` tool to add one to 55.3, resulting in 56.3.', tool_calls=None)

In [34]:
print(response_message.content)

I used the `add_one` tool to add one to 55.3, resulting in 56.3.
